In [13]:
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.callbacks import TensorBoard
import pandas as pd
import numpy as np
from datetime import datetime
import tensorflow as tf

In [14]:
# model
class MLP(tf.keras.Model):
    def __init__(self):
        super(MLP, self).__init__()
        self.dense1 = tf.keras.layers.Dense(512, activation="relu")
        self.dense2 = tf.keras.layers.Dense(512, activation="relu")
        self.dropout1 = tf.keras.layers.Dropout(0.1)
        self.dense3 = tf.keras.layers.Dense(1028, activation="relu")
        self.dense4 = tf.keras.layers.Dense(1, activation="relu")

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dropout1(x)
        x = self.dense3(x)
        return self.dense4(x)

In [15]:
train = pd.read_csv("prep_train_onehot.csv")
test = pd.read_csv("prep_test_onehot.csv")
df = train["y"]
predata = pd.concat([train.drop("y", axis=1), test])

/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3,10,12,18,20,26,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/macuser/anaconda3/envs/salary_pred/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (7,9,25,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
predata = predata.drop(['Type','Region','MunicipalityCode','Prefecture','Municipality','DistrictName','NearestStation',
            'FloorPlan','LandShape','Structure','Use','Purpose','Classification','CityPlanning', 'Direction',
            'Renovation','Remarks','era_name'], axis=1)

,y,id,Type,Region,MunicipalityCode,Prefecture,Municipality,DistrictName,NearestStation,TimeToNearestStation,...,Remarks_関係者間取引、調停・競売等、私道を含む取引,Remarks_隣地の購入,Remarks_隣地の購入、その他事情有り,Remarks_隣地の購入、瑕疵有りの可能性,Remarks_隣地の購入、私道を含む取引,Remarks_隣地の購入、調停・競売等,Remarks_隣地の購入、調停・競売等、私道を含む取引,Remarks_隣地の購入、関係者間取引,Remarks_隣地の購入、関係者間取引、私道を含む取引,era_name_昭和
0,66.0,1,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,1.0,...,0,0,0,0,0,0,0,0,0,1
1,19.0,2,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,5.0,...,0,0,0,0,0,0,0,0,0,0
2,37.0,3,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,3.0,...,0,0,0,0,0,0,0,0,0,0
3,18.0,4,中古マンション等,NaN,13101,東京都,千代田区,飯田橋,飯田橋,5.0,...,0,0,0,0,0,0,0,0,0,0
4,240.0,5,宅地(土地と建物),商業地,13101,東京都,千代田区,飯田橋,飯田橋,3.0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
train[["y", "id", "TimeToNearestStation", "TotalFloorArea", "Area", "Frontage", "BuildingYear", "Direction", "Breadth", "CoverageRatio", "FloorAreaRatio", "Period"]].isnull().sum()

y                            0
id                           0
TimeToNearestStation     10129
TotalFloorArea          236201
Area                      1305
Frontage                177161
BuildingYear             82841
Direction               159830
Breadth                 163065
CoverageRatio             5386
FloorAreaRatio            5386
Period                       0
dtype: int64

In [8]:
# 0 ~ 1 の範囲に正則化
def min_max(x):
    min_value = x.min()
    max_value = x.max()
    result = (x - min_value)/(max_value - min_value)
    return result

num_list = [
    "TimeToNearestStation", "TotalFloorArea", "Area", "Frontage", "BuildingYear", "BuildingAge", 
    "Breadth", "CoverageRatio", "FloorAreaRatio", "Period"
    ]
for num in num_list:
    predata[num] = min_max(predata[num])
    
   

TypeError: '<=' not supported between instances of 'str' and 'float'

In [7]:
#train test split
prep_train = predata.iloc[:len(train), :]
prep_test = predata.iloc[len(train):, :]

x_train = prep_train.values
y_train = df.values
x_test = prep_test

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, rabdom_state=71)



ValueError: nan is not in list

In [0]:
# train
model = MLP()
tensorboard = TensorBoard(log_dir="logs")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    "./checkpoint/MLP-{epoch:04d}.ckpt",
    verbose=1,
    save_weights_only=True,
    period=500
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-1),
    loss='mean_squared_error',
    metrics=[
        "mean_squared_error", 
 #       "mean_absolute_error",
 #       "mean_squared_error"
    ]
)
model.fit(
    x_train, 
    y_train, 
    epochs=100,
    batch_size=512,
    validation_data=(x_valid, y_valid),
    callbacks=[tensorboard, checkpoint]
)

Train on 30000 samples, validate on 5000 samples
Epoch 1/100
30000/30000 [==============================] - 3s 94us/sample - loss: 63.4185 - mean_absolute_percentage_error: 63.4185 - val_loss: 29.3284 - val_mean_absolute_percentage_error: 29.3284
Epoch 2/100
30000/30000 [==============================] - 3s 89us/sample - loss: 13.7878 - mean_absolute_percentage_error: 13.7878 - val_loss: 9.4491 - val_mean_absolute_percentage_error: 9.4491
Epoch 3/100
30000/30000 [==============================] - 3s 89us/sample - loss: 9.5135 - mean_absolute_percentage_error: 9.5135 - val_loss: 9.2566 - val_mean_absolute_percentage_error: 9.2566
Epoch 4/100
30000/30000 [==============================] - 3s 89us/sample - loss: 9.2067 - mean_absolute_percentage_error: 9.2067 - val_loss: 8.8954 - val_mean_absolute_percentage_error: 8.8954
Epoch 5/100
30000/30000 [==============================] - 3s 88us/sample - loss: 8.9930 - mean_absolute_percentage_error: 8.9930 - val_loss: 9.5917 - val_mean_absolute_

In [0]:
train_mse, train_acc = model.evaluate(x_train,  y_train)
val_mse, val_acc = model.evaluate(x_valid,  y_valid)
print(f'train rmse : {np.sqrt(train_rmse):.3f}%' )

print(f'valid rmse : {np.sqrt(val_rmse):.3f}%' )

5000/5000 [==============================] - 0s 77us/sample - loss: 7.6253 - mean_absolute_percentage_error: 7.6253
train mape : 6.573%
valid mape : 7.625%


In [0]:
model.dropout1 = tf.keras.layers.Dropout(0.0)

model.predict_on_batch(x_test)

array([[19491340.]], dtype=float32)

In [0]:
model.dropout1 = tf.keras.layers.Dropout(0.0)
data = data.append(pd.DataFrame.from_dict({
    "駅名": ["新橋"],
    "建物構造": ["SRC"],
    "徒歩分": [10],
    "専有面積": [30],
    "間取り": ["R"],
    "部屋数": [1],
    "築年": ["198001"],
    "所在階": ["5"],
    "成約価格": [-1]
})).reset_index(drop=True)
X = create_feature(data)
model.predict_on_batch(X[-1:])

array([[19491340.]], dtype=float32)